In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
torch.autograd.set_detect_anomaly(True)

from torch_geometric.data import Data
from src.tgnn import TGN

In [14]:
"""
Attributes
----------


Questions
---------
- Are edges directional? If yes, do we create (i,j) and (j,i)?
- Do we aggregate node_wise_event-messages and interaction_event-messages together, if yes how, because different dim?
- How do we make sure, that the concatenation of all neighbours in GAT is equally long, if node has few neighbours? 
  Just fill up to standard length with zeros?
- what happens to nodes with degree 0 in the GAT layer? Just propagate the unmodified h = s+v through all layers?
- I've modelled v_i = 0 as a deleted node and e_ij = 0 as a deleted edge and e_ij = 1 as a existing edge. 
  Is this good, or is it better allow nodes and edges to have all values (including 0)
"""

d_t = 1

d_v = 2
d_e = 1
d_h = d_v
d_s = d_v

d_rm_v = d_s + 1 + d_v
d_rm_e = 2*d_s + 1 + d_e
d_m = 2

l = 1
k = 1
d_q = d_h + d_t
d_c = d_h + d_t + d_e
n = 3

TGN_hyperparams = dict()
TGN_hyperparams['d_t'] = 1
TGN_hyperparams['d_v'] = 2
TGN_hyperparams['d_e'] = 1
TGN_hyperparams['d_m'] = 2
TGN_hyperparams['num_m'] = 3
TGN_hyperparams['l'] = 1
TGN_hyperparams['k'] = 1
TGN_hyperparams['n'] = 3

TGN_hyperparams['d_h'] = TGN_hyperparams['d_v']
TGN_hyperparams['d_s'] = TGN_hyperparams['d_v']
TGN_hyperparams['d_rm_v'] = TGN_hyperparams['d_s'] + 1 + TGN_hyperparams['d_v']
TGN_hyperparams['d_rm_e'] = 2*TGN_hyperparams['d_s'] + 1 + TGN_hyperparams['d_e']
TGN_hyperparams['d_q'] = TGN_hyperparams['d_h'] + TGN_hyperparams['d_t']
TGN_hyperparams['d_c'] = TGN_hyperparams['d_h'] + TGN_hyperparams['d_t'] + TGN_hyperparams['d_e']




"""
Initialize Graph
----------------
N_0: initial number of nodes in the graph
t_0: initial time
"""
N_0 = 3
t_0 = torch.tensor([0.])

v = torch.tensor([[0., 0.], [10., 10.], [20., 20.]])
e_id = torch.tensor([[0, 1], [2, 1]]).T
e = torch.tensor([[1.], [1.]])

G = Data(v, e_id, e)
G

"""
New events
"""
t_1 = torch.tensor([1.])
v_i = torch.tensor([30., 30.])
node_wise_events = [(3, v_i, t_1)]

e_ij = torch.tensor([1.])
t_2 = torch.tensor([1.])
interaction_events = [((1,0), e_ij, t_2)]


In [15]:
"""
Model
"""
model = TGN(G, TGN_hyperparams)
optimizer = Adam(model.parameters(), lr = 0.001)